In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df

from scipy import stats

import sklearn
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

In [22]:
train_x = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_x= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')

In [13]:
train_x.head()

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,0,2018-01-01,서귀포시,남원읍,291408.897,18744.131,219588.857,529741.884,53.000,40.571,2.35,0.0,3.325
1,1,2018-01-01,제주시,노형동,1215420.248,74964.858,445735.329,1736120.435,38.563,30.655,-2.30,0.0,1.100
2,2,2018-01-01,제주시,구좌읍,276220.023,17542.105,248104.122,541866.250,53.188,40.719,5.90,0.0,4.100
3,3,2018-01-01,제주시,용담1동,159129.790,9363.622,101939.586,270432.998,35.833,14.000,2.90,0.0,2.400
4,4,2018-01-01,제주시,건입동,174544.301,19772.091,191560.904,385877.295,33.600,43.400,0.80,0.0,2.600


In [17]:
train_y.head()

,id,교통량
0,0,76.200
1,1,1291.184
2,2,0.000
3,3,610.167
4,4,393.400


각데이터의 결측치, 데이터타입, 연속형변수의 분포를 파악하라

In [11]:
missing_cols = []
for head in test_x.keys():
    if test_x[head].isnull().sum() != 0:
        missing_cols.append(head)
print(missing_cols)

[]


LabelEncoder를 이용하여 trainset의 읍면동, 시도명을 fit하고 train_X,test_X를 각각 transform 하여 pre_train, pre_test 변수에 저장하고 상위 5개값을 출력하라

In [40]:
obj_cols = ['읍면동명', '시도명']
pre_train = train_x.copy()
pre_test = test_x.copy()

for head in obj_cols:
    encoder = LabelEncoder()
    pre_train[head] = encoder.fit_transform(train_x[head])
    encoder = LabelEncoder()
    pre_test[head] = encoder.fit_transform(test_x[head])

‘id’ 컬럼을 제거하라. ‘일자’를 년도,월,일,요일(숫자)로 각각 분리하여 컬럼을 형성하라. 그후 ‘일자’컬럼을 제거하라

In [41]:
pre_train = pre_train.drop(['id'], axis=1)
pre_test = pre_test.drop(['id'], axis=1)

pre_train['년도'] = 2018
pre_test['년도'] = 2018
pre_train['월'] = 1
pre_test['월'] = 1
pre_train['일'] = 1
pre_test['일'] = 1

for i, value in enumerate(pre_train['일자']):
    times = str(value).split('-')
    pre_train['년도'][i] = int(times[0])
    pre_train['월'][i] = int(times[1])
    pre_train['일'][i] = int(times[2])

for i, value in enumerate(pre_test['일자']):
    times = str(value).split('-')
    pre_test['년도'][i] = int(times[0])
    pre_test['월'][i] = int(times[1])
    pre_test['일'][i] = int(times[2])
    
pre_train = pre_train.drop(['일자'],axis=1)
pre_test = pre_test.drop(['일자'],axis=1)
pre_train.head()

<ipython-input-41-1f49afc9a85d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train['년도'][i] = int(times[0])
<ipython-input-41-1f49afc9a85d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train['월'][i] = int(times[1])
<ipython-input-41-1f49afc9a85d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train['일'][i] = int(times[2])
<ipython-input-41-1f49afc9a85d>:19: SettingWithCopyWarning: 
A value is trying to be set o

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,년도,월,일
0,0,2,291408.897,18744.131,219588.857,529741.884,53.000,40.571,2.35,0.0,3.325,2018,1,1
1,1,3,1215420.248,74964.858,445735.329,1736120.435,38.563,30.655,-2.30,0.0,1.100,2018,1,1
2,1,1,276220.023,17542.105,248104.122,541866.250,53.188,40.719,5.90,0.0,4.100,2018,1,1
3,1,24,159129.790,9363.622,101939.586,270432.998,35.833,14.000,2.90,0.0,2.400,2018,1,1
4,1,0,174544.301,19772.091,191560.904,385877.295,33.600,43.400,0.80,0.0,2.600,2018,1,1


인구 관련 컬럼(거주인구,근무인구,방문인구,총 유동인구를 정규화하라. 정규화시 trainset에 대한 fit을 test셋에 적용하라.

In [42]:
pop_cols = ['거주인구', '근무인구', '방문인구', '총 유동인구']
for head in pop_cols:
    scaler = StandardScaler()
    np_pop = np.array(pre_train[head]).reshape(-1,1)
    scaler.fit(np_pop)
    pre_train[head] = scaler.transform(np_pop)
    np_pop_test = np.array(pre_test[head]).reshape(-1,1)
    pre_test[head] = scaler.transform(np_pop_test)
pre_train.head()

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,년도,월,일
0,0,2,-0.087699,-0.418008,0.166825,-0.042833,53.000,40.571,2.35,0.0,3.325,2018,1,1
1,1,3,3.011526,0.959452,1.775292,2.573146,38.563,30.655,-2.30,0.0,1.100,2018,1,1
2,1,1,-0.138644,-0.447459,0.369639,-0.016541,53.188,40.719,5.90,0.0,4.100,2018,1,1
3,1,24,-0.531376,-0.647839,-0.669956,-0.605133,35.833,14.000,2.90,0.0,2.400,2018,1,1
4,1,0,-0.479674,-0.392822,-0.032524,-0.354797,33.600,43.400,0.80,0.0,2.600,2018,1,1


In [43]:
pre_test.head()

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,년도,월,일
0,1,7,-0.956943,-0.793506,-0.664483,-0.891489,39.556,29.167,5.0,0.0,2.5,2018,1,1
1,1,23,-0.349010,-0.623888,-0.593474,-0.461793,32.900,30.900,5.0,0.0,2.5,2018,1,1
2,1,27,3.001336,1.497394,2.454246,2.821168,29.538,35.692,2.9,0.0,2.4,2018,1,1
3,1,29,-0.951102,-0.729412,-0.881793,-0.948293,30.000,23.500,2.9,0.0,2.4,2018,1,1
4,0,6,-0.545104,-0.646851,-0.321850,-0.507790,41.583,14.375,5.1,0.0,2.3,2018,1,1


적당한 회귀 모델을 선정하고, 학습 및 mean_squared_error 값을 통해 모델을 검증하라

In [57]:
from sklearn.metrics import mean_squared_error

t_x, v_x, t_y, v_y = train_test_split(pre_train, train_y['교통량'], test_size=0.2, random_state=1)

model_1 = LinearRegression()
model_1.fit(t_x, t_y)
prediction = model_1.predict(v_x)
score = mean_squared_error(v_y, prediction)
print('linear ', score)

linear  199310.35172042492


## My own methods

In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [106]:
input_df = train_x.copy()
input_df = input_df.drop(['id'],axis=1)
## 근무인구, 일강수량, 평균 풍속
outlier_cols = ['근무인구', '일강수량', '평균 풍속']
for head in outlier_cols:
    Q1 = input_df[head].quantile(.25)
    Q3 = input_df[head].quantile(.75)
    IQR = Q3 - Q1
    up_bound = Q3 + IQR*3/2
    dw_bound = Q1 - IQR*3/2
    input_df.loc[(input_df[head] > up_bound), head] = up_bound
    if dw_bound < 0:
        input_df.loc[(input_df[head] < dw_bound), head] = 0
    else:
        input_df.loc[(input_df[head] < dw_bound), head] = dw_bound

test_df = test_x.copy()
test_df = test_df.drop(['id'], axis=1)
input_df = input_df.drop(['일자'], axis=1)
test_df = test_df.drop(['일자'], axis=1)

obj_cols = ['시도명', '읍면동명']
num_cols = [head for head in input_df.keys() if head not in obj_cols]

num_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())])
obj_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('obj', obj_transformer, obj_cols)])

seed = 1
K_val = 5; scoring='neg_mean_squared_error'; num_repeat=1;
cv = RepeatedStratifiedKFold(n_splits=K_val, n_repeats=num_repeat, random_state=seed)

t_x, v_x, t_y, v_y = train_test_split(input_df, train_y['교통량'], test_size=0.2, random_state=1)

clf_1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linear', LinearRegression())])

clf_2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('NN', MLPRegressor())])

clf_3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('SVR', SVR())])

clf_4 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('RF', RandomForestRegressor())])

models = [('linear', clf_1), ('NN',clf_2), ('SVR',clf_3), ('RF', clf_4)]
names = []
means = []
for name, model in models:
    model.fit(t_x, t_y)
    prediction = model.predict(v_x)
    result = mean_squared_error(v_y, prediction)
    names.append(name)
    means.append(result)
result_df = df({'name':names, 'result':means})
result_df

C:\Users\fogbo\anaconda3\envs\first_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,name,result
0,linear,24654.314451
1,NN,22584.147330
2,SVR,238679.133792
3,RF,9414.365444


In [114]:
RF = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('RF', RandomForestRegressor())])
RF_param_grid ={
    'RF__n_estimators':[10, 100],
    'RF__max_features':['auto', 'sqrt', 'log2']}
    
RF_grid = GridSearchCV(RF, param_grid=RF_param_grid)
RF_grid.fit(t_x,t_y)
RF_best = RF_grid.best_estimator_
RF_grid.best_score_

0.9658155816009704

In [140]:
v_x

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
8451,제주시,이도2동,1187697.488,82459.460125,538613.579,1816250.806,29.115,37.000,9.500,0.000,2.500
444,서귀포시,중문동,158235.000,15618.980000,131216.768,305070.748,48.278,44.000,1.500,0.000,1.300
11912,서귀포시,대륜동,295524.298,37997.664000,198481.412,532003.374,39.000,32.481,15.500,0.000,2.000
19727,서귀포시,남원읍,294458.156,22688.265000,160358.383,477504.803,49.065,49.717,9.625,0.000,2.100
14441,서귀포시,남원읍,283875.794,28063.987000,177806.027,489745.808,49.674,49.478,21.425,3.750,2.225
...,...,...,...,...,...,...,...,...,...,...,...
10244,제주시,조천읍,412742.290,44256.167000,265599.425,722597.881,51.771,46.543,2.900,0.667,2.600
636,제주시,삼도2동,100005.920,13683.430000,143035.497,256724.846,27.667,17.667,-4.300,0.000,3.300
3773,서귀포시,중문동,156310.889,21651.069000,121644.450,299606.408,48.833,44.611,14.100,3.750,1.300
18297,서귀포시,안덕면,411430.335,60021.272000,188021.143,659472.750,47.730,47.919,7.500,0.000,2.800


In [142]:
test_x

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,22448,2018-01-01,제주시,도두동,32249.987,3418.266,102709.092,138377.345,39.556,29.167,5.0,0.0,2.50
1,22449,2018-01-01,제주시,외도동,213500.997,10341.172,112692.789,336534.958,32.900,30.900,5.0,0.0,2.50
2,22450,2018-01-01,제주시,이도2동,1212382.218,96920.834,541194.481,1850497.533,29.538,35.692,2.9,0.0,2.40
3,22451,2018-01-01,제주시,일도1동,33991.653,6034.253,72155.919,112181.825,30.000,23.500,2.9,0.0,2.40
4,22452,2018-01-01,서귀포시,대천동,155036.925,9403.969,150882.409,315323.303,41.583,14.375,5.1,0.0,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9616,32064,2020-04-30,서귀포시,표선면,228260.005,16891.732,152832.449,397984.186,44.514,34.054,18.1,0.0,4.70
9617,32065,2020-04-30,제주시,한림읍,459959.064,26007.122,249700.419,735666.605,48.609,61.377,18.0,0.0,4.35
9618,32066,2020-04-30,제주시,도두동,28397.481,3144.895,84052.697,115595.073,41.053,29.421,20.3,0.0,3.00
9619,32067,2020-04-30,서귀포시,안덕면,348037.846,29106.286,251129.660,628273.792,46.595,49.189,17.6,0.0,3.50


In [143]:
prediction = RF_best.predict(v_x)
final_score = mean_squared_error(v_y, prediction)
print(final_score)

input_test = test_x.drop(['id','일자'], axis=1)
prediction = RF_best.predict(input_test)
submit_df = df({'id':test_x['id'],'교통량':prediction})

9458.49881692537


In [144]:
submit_df

,id,교통량
0,22448,1420.16558
1,22449,1127.85058
2,22450,324.41040
3,22451,0.00000
4,22452,0.74138
...,...,...
9616,32064,1.01275
9617,32065,0.00000
9618,32066,1532.29685
9619,32067,1.09815
